# Observable Trends








* Based on the data below, it is clear that Charter schools outperform District schools in the delivery of academic results.  Charter schools are smaller in size and have greater disparity in their spend per student, ranging between AUD578-AUD638 per student as compared with District schools that spent between AUD628-AUD655 per student.

* Despite this however at the highest level, comparing average scores in maths and reading, as well as pass rates in one or both subjects, Charter schools deliver consistently higher results than District schools.  This observation also holds when drilling down into results for not only each subject but also across the grades (Grades 9 to Grade 12 inclusive).



### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

In [2]:
# File to Load (Remember to Change These)
school_data_csv = "Resources/schools_complete.csv"

# Dependencies and Setup
student_data_csv = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data_df = pd.read_csv(school_data_csv)
student_data_df = pd.read_csv(student_data_csv)

In [3]:
# Combine the data into a single dataset.  
all_school_data_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
all_school_data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

*** Column "size" = number of students.  Logic for Budget calculation is due to duplication in the series.  Many:1 relationship budget to "Student ID" (and subsequently "school_name").  Code aggregated into sections: create dataframe -> perform calculations -> generate report -> reformat and display report.

In [4]:
# Reduce dataframe size
columns = ["school_name","type","budget","math_score","reading_score","size"]
district_data_df = all_school_data_df.loc[:,columns]


In [5]:
# Calculate counts (school and students)
school_count = len(district_data_df["school_name"].unique())
student_count = district_data_df["size"].count()


# Calculate total budget for all districts
budgets = district_data_df["budget"].unique()  # unique budget value array
total_budget = budgets.sum()


# Calculate score averages (math and reading)
avg_math_score = district_data_df["math_score"].mean()
avg_reading_score = district_data_df["reading_score"].mean()


# Calculate percentages passing (math only)
pass_math_count = district_data_df["math_score"].loc[district_data_df["math_score"] >= 70].count()
percent_pass_math = pass_math_count / student_count * 100


# Calculate percentages passing (reading only)
pass_reading_count = district_data_df["reading_score"].loc[district_data_df["reading_score"] >= 70].count()
percent_pass_reading = pass_reading_count / student_count * 100


# Calculate percentages passing (both math and reading)
pass_both_df = district_data_df.loc[(district_data_df["math_score"] >= 70) &
                                 (district_data_df["reading_score"] >= 70)
                                ].count()

percent_pass_both = pass_both_df["math_score"] / student_count * 100


In [6]:
# Create a summary DataFrame using above objects
district_summary_df = pd.DataFrame({"Total Schools": [school_count],
                           "Total Students": [student_count],
                           "Total Budget": [total_budget],
                           "Average Math Score": [avg_math_score],
                           "Average Reading Score": [avg_reading_score],
                           "% Passing Math": [percent_pass_math],
                           "% Passing Reading": [percent_pass_reading],
                           "% Overall Passing": [percent_pass_both]
                                   }
                                  )


In [7]:
# Reformat and display summary report
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:,.2f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.2f}".format)

district_summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

*** Column "school_name" renamed at beginning as unable to relabel index once groupby function used.  Code aggregated into sections: create dataframe -> perform calculations -> generate, reformat and display report.

In [8]:
# Reduce dataframe size
columns = ["school_name","type","budget","math_score","reading_score","size"]
school_data_df = all_school_data_df.loc[:,columns]
new_school_df = school_data_df.rename(columns={"school_name":"School Name"})


*** Use of unique aggregation created string of strings series for "school_type" ie inserted [] into the field values.  Required to remove from column (refer remark below) for 'clean' display

In [9]:
# Calculate counts (school and students)
school_type = new_school_df.groupby("School Name").type.unique()
school_type = school_type.str.get(0) # converted to an integer
total_students = new_school_df.groupby("School Name").math_score.count()


# Calculate percentages passing (math only)
math_pass_df = new_school_df.loc[new_school_df["math_score"] >= 70]
group_math_pass = math_pass_df.groupby("School Name").math_score.count()
passing_math = group_math_pass / total_students * 100


# Calculate percentages passing (reading only)
reading_pass_df = new_school_df.loc[new_school_df["reading_score"] >= 70]
group_reading_pass = reading_pass_df.groupby("School Name").reading_score.count()
passing_reading = group_reading_pass / total_students * 100


# Calculate percentages passing (both math and reading)
both_pass_df = new_school_df.loc[(new_school_df["reading_score"] >= 70) & 
             (new_school_df["math_score"] >= 70)]
group_both_pass = both_pass_df.groupby("School Name").math_score.count()
passing_both = group_both_pass / total_students * 100


*** Created new dataframe to use aggregation functions efficiently (ie required averages for "budget", "math_score", "reading_score" and "size").  Creation reduces code to a single line (refer remark below).

In [10]:
# Create, reformat and display summary report
school_summary_df = new_school_df.groupby(["School Name"]).mean() #groupby and aggregation brings back dataframe means
school_summary_df["School Type"] = school_type

student_budget = school_summary_df["budget"] / school_summary_df["size"]
school_summary_df["Per Student Budget"] = student_budget
school_summary_df["% Passing Math"] = passing_math
school_summary_df["% Passing Reading"] = passing_reading
school_summary_df["% Overall Passing"] = passing_both


# Reorder columns
school_summary2_df = school_summary_df.reindex(columns=["School Type","size","budget", "Per Student Budget", "math_score", 
                                                  "reading_score", "% Passing Math","% Passing Reading",
                                                  "% Overall Passing"
                                                 ]
                                        )


#Use .rename(columns={}) in order to rename columns
school_summary3_df = school_summary2_df.rename(columns={"school_name":"School Name", "size":"Total Students", 
                                                  "math_score": "Average Math Score","budget":"Total School Budget",
                                                  "reading_score": "Average Reading Score"
                                                 }
                                        )


# Reformat output for display
school_summary3_df["Total School Budget"] = school_summary3_df["Total School Budget"].map("${:,.2f}".format)
school_summary3_df["Total Students"] = school_summary3_df["Total Students"].map("{:,.0f}".format)
school_summary3_df["Per Student Budget"] = school_summary3_df["Per Student Budget"].map("${:.2f}".format)
school_summary3_df["Average Math Score"] = school_summary3_df["Average Math Score"].map("{:.2f}".format)
school_summary3_df["Average Reading Score"] = school_summary3_df["Average Reading Score"].map("{:.2f}".format)
school_summary3_df["% Passing Math"] = school_summary3_df["% Passing Math"].map("{:.2f}".format)
school_summary3_df["% Passing Reading"] = school_summary3_df["% Passing Reading"].map("{:,.2f}".format)
school_summary3_df["% Overall Passing"] = school_summary3_df["% Overall Passing"].map("{:.2f}".format)

school_summary3_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [11]:
best_schools = school_summary3_df.sort_values("% Overall Passing", ascending=False)
best_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [12]:
best_schools = school_summary3_df.sort_values("% Overall Passing")
best_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

*** Created new dataframe to use aggregation functions more efficiently.  As per instructions created individual series using conditional statement "isin".
Column "school_name" renamed at beginning as unable to relabel index once groupby function used.  Code aggregated into sections: create dataframe -> perform calculations -> reformat and display report.

In [13]:
# Reduce dataframe size
columns = ["school_name","grade","math_score"]
grade_df = all_school_data_df.loc[:,columns]
new_grade_df = grade_df.rename(columns={"school_name":"School Name"})


In [14]:
# Segment math scores for each grade
math_grade9_df = new_grade_df[new_grade_df["grade"].isin(["9th"])]
math_grade10_df = new_grade_df[new_grade_df["grade"].isin(["10th"])]
math_grade11_df = new_grade_df[new_grade_df["grade"].isin(["11th"])]
math_grade12_df = new_grade_df[new_grade_df["grade"].isin(["12th"])]


# Create sequence of series to group segmented math scores for each grade by school
math_grouped_grade9s = math_grade9_df.groupby("School Name").math_score.mean()
math_grouped_grade10s = math_grade10_df.groupby("School Name").math_score.mean()
math_grouped_grade11s = math_grade11_df.groupby("School Name").math_score.mean()
math_grouped_grade12s = math_grade12_df.groupby("School Name").math_score.mean()


# Merge series of math scores for each grade by school into one dataframe
math_merge_df = pd.merge(math_grouped_grade9s, math_grouped_grade10s, on="School Name")
math_merge_df = pd.merge(math_merge_df, math_grouped_grade11s, on="School Name")
math_merge_df = pd.merge(math_merge_df, math_grouped_grade12s, on="School Name")



In [15]:
# Use .columns for renaming columns in a dataframe
math_merge_df.columns = ["9th", "10th", "11th", "12th"] 

# Reformat output for display
math_merge_df["9th"] = math_merge_df["9th"].map("{:,.2f}".format)
math_merge_df["10th"] = math_merge_df["10th"].map("{:,.2f}".format)
math_merge_df["11th"] = math_merge_df["11th"].map("{:,.2f}".format)
math_merge_df["12th"] = math_merge_df["12th"].map("{:,.2f}".format)

math_merge_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

*** Same code as for math.  Updated for reading column.

In [16]:
# Reduce dataframe size
columns = ["school_name","grade","reading_score"]
grade_df = all_school_data_df.loc[:,columns]
new_grade_df = grade_df.rename(columns={"school_name":"School Name"})


# Segment reading scores for each grade
reading_grade9_df = new_grade_df[new_grade_df["grade"].isin(["9th"])]
reading_grade10_df = new_grade_df[new_grade_df["grade"].isin(["10th"])]
reading_grade11_df = new_grade_df[new_grade_df["grade"].isin(["11th"])]
reading_grade12_df = new_grade_df[new_grade_df["grade"].isin(["12th"])]


In [17]:
# Create sequence of series to group segmented reading scores for each grade by school
reading_grouped_grade9s = reading_grade9_df.groupby("School Name").reading_score.mean()
reading_grouped_grade10s = reading_grade10_df.groupby("School Name").reading_score.mean()
reading_grouped_grade11s = reading_grade11_df.groupby("School Name").reading_score.mean()
reading_grouped_grade12s = reading_grade12_df.groupby("School Name").reading_score.mean()


In [18]:
# Merge series of reading scores for each grade by school into one dataframe
reading_merge_df = pd.merge(reading_grouped_grade9s, reading_grouped_grade10s, on="School Name")
reading_merge_df = pd.merge(reading_merge_df, reading_grouped_grade11s, on="School Name")
reading_merge_df = pd.merge(reading_merge_df, reading_grouped_grade12s, on="School Name")


# Use .columns for renaming columns in a dataframe
reading_merge_df.columns = ["9th", "10th", "11th", "12th"] 


# Reformat output for display
reading_merge_df["9th"] = reading_merge_df["9th"].map("{:,.2f}".format)
reading_merge_df["10th"] = reading_merge_df["10th"].map("{:,.2f}".format)
reading_merge_df["11th"] = reading_merge_df["11th"].map("{:,.2f}".format)
reading_merge_df["12th"] = reading_merge_df["12th"].map("{:,.2f}".format)

reading_merge_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

*** To minimise code, an already existing dataframe "school_summary2_df" has been with removal of columns not required prior to report displays.  This dataframe was selected as "school_summary3_df" includes reformatted columns (values have been converted to strings).

An alternate approach would have been to the insert code used for the "School Summary" task to generate the school_summary2_df.  Given the iterative nature of scores analysis tasks, my approach would remain to generate a base dataframe and then reformat as required.  With this approach the only columns in the dataframe "school_summary3_df" redundant to the combined 3 tasks are the index "School Name" and "budget".  These would not be removed from the dataframe to preserve 'readability' in debugging.

NB: Below requires prior execution of "School Summary" code to generate starting dataframe called.

In [19]:
# Reduce dataframe size
columns = ["Per Student Budget","math_score","reading_score","% Passing Math","% Passing Reading","% Overall Passing"]
size_df = school_summary2_df.loc[:,columns]


#Use .rename(columns={}) in order to rename columns
size2_df = size_df.rename(columns={"math_score": "Average Math Score","reading_score": "Average Reading Score"
                                              }
                                     )



In [20]:
# Create the bins in which Data will be held
# Bins are 0, 584, 629, 644, 675   
bins = [0, 584, 629, 644, 675]

# Create labels for the bins
group_labels = ["<$584", "$585-629", "$630-644", "$645-675"]

# Slice the data and place it into bins using Summary report (prior to reformatting and conversion to strings)
pd.cut(size2_df["Per Student Budget"], bins, labels=group_labels).head()

# Place the data series into a new column inside of the DataFrame
size2_df["Spending Ranges (Per Student)"] = pd.cut(size2_df["Per Student Budget"], bins, labels=group_labels)


In [21]:
# Remove bin column from dataframe
del size2_df['Per Student Budget']

spend_group = size2_df.groupby("Spending Ranges (Per Student)")
spend_group.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-644,78.518855,81.624473,73.484209,84.391793,62.857656
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

*** To minimise code, an already existing dataframe "school_summary2_df" has been with removal of columns not required prior to report displays.  This dataframe was selected as "school_summary3_df" includes reformatted columns (values have been converted to strings).

An alternate approach would have been to the insert code used for the "School Summary" task to generate the school_summary2_df.  Given the iterative nature of scores analysis tasks, my approach would remain to generate a base dataframe and then reformat as required.  With this approach the only columns in the dataframe "school_summary3_df" redundant to the combined 3 tasks are the index "School Name" and "budget".  These would not be removed from the dataframe to preserve 'readability' in debugging.

NB: Below requires prior execution of "School Summary" code to generate starting dataframe called.

In [22]:
# Reduce dataframe size
columns = ["size","math_score","reading_score","% Passing Math","% Passing Reading","% Overall Passing"]
size_df = school_summary2_df.loc[:,columns]


#Use .rename(columns={}) in order to rename columns
size2_df = size_df.rename(columns={"math_score": "Average Math Score","reading_score": "Average Reading Score"
                                              }
                                     )



In [23]:
# Create the bins in which Data will be held
# Bins are 0, 1000, 2000, 5000   
bins = [0, 1000, 2000, 5000]


# Create labels for these bins
group_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


# Slice the data and place it into bins
pd.cut(size2_df["size"], bins, labels=group_labels).head()


# Place the data series into a new column inside of the DataFrame
size2_df["School Size"] = pd.cut(size2_df["size"], bins, labels=group_labels)



In [24]:
# Remove bin column from dataframe
del size2_df['size']


# Group dataframe by size
size_group_df = size2_df.groupby("School Size")


size_group_df.mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [25]:
# Reduce dataframe size
columns = ["School Type","math_score","reading_score","% Passing Math","% Passing Reading","% Overall Passing"]
size_df = school_summary2_df.loc[:,columns]


#Use .rename(columns={}) in order to rename columns
size2_df = size_df.rename(columns={"math_score": "Average Math Score","reading_score": "Average Reading Score"
                                              }
                                     )


In [26]:
school_type = size2_df.groupby("School Type")
school_type.mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
